In [1]:
import pandas as pd
import numpy as np
#import plotly.io as pio
#import plotly.express as px
#pd.options.plotting.backend = "plotly"

In [2]:
#Path
path = '/Users/jpmvbastos/Library/CloudStorage/OneDrive-TexasTechUniversity/Fall 2023/Causal Inference/Term Paper/Data/RAIS/'

#Loop to read each year  
start_year = 2006
end_year = 2017
years = list(range(start_year, end_year + 1))

# Empty lists to collect the name of the dfs
names_list = []

# The globals() method is necessary to name the dfs using the string generated in the loop
for year in years:
    print('Building year ' + str(year))
    name = f'raisestab{year}'  
  #Open file
    df = pd.read_csv(path+f'ESTB{year}.txt', delimiter=';', encoding='latin1', low_memory=False)
  
    df[df['Ind Atividade Ano']==1] #Exclude inactive firms 
    df['year'] = year # Create year var for panel structure 
  
 # Get the CNAE codes 
    df['CNAE2'] = df['CNAE 2.0 Subclasse'].astype(str).apply(lambda x: int(x[:-5]))

# Type of Establishment
    df['Natureza Jurídica'] = pd.to_numeric(df['Natureza Jurídica'], errors='coerce')
    df['government'] = np.where(df['Natureza Jurídica'] < 2000, 1, 0)
    df[df['government']==0] #focus on private firms
# Private Companies and Associations
    df['private'] = np.where((df['Natureza Jurídica'] > 2011) & (df['Natureza Jurídica'] < 2292),1,0)
# Individual companies
    df['individual'] = np.where(df['Tipo Estab']==3, 1, 0)

 ## Firm Size
# Define the conditions and choice
    df['small'] = np.where(df['Tamanho Estabelecimento'] < 5, 1, 0)
    df['medium'] = np.where((df['Tamanho Estabelecimento'] > 4) 
                                & (df['Tamanho Estabelecimento'] < 7), 1, 0)
    df['large'] = np.where(df['Tamanho Estabelecimento'] > 6, 1, 0)

    # Create 'transportation' variable
    df['transportation'] = 0
    df.loc[df['CNAE2'].notnull(), 'transportation'] = np.where(df['CNAE2'].isin([49, 50, 51, 52, 53, 61, 79]), 1, 0)

# Create 'accommodation' variable
    df['accommodation'] = 0
    df.loc[df['CNAE2'].notnull(), 'accommodation'] = np.where(df['CNAE2'].isin([55, 56, 59, 60, 90, 91, 92, 93, 94]), 1, 0)

# Create 'retail' variable
    df['retail'] = 0
    df.loc[df['CNAE2'].notnull(), 'retail'] = np.where(df['CNAE2'] == 47, 1, 0)

    # Create 'construction' variable
    df['construction'] = 0
    df.loc[df['CNAE2'].notnull(), 'construction'] = np.where(df['CNAE2'].isin([41, 42, 43]), 1, 0)

    sectors = ['transportation', 'accommodation', 'retail', 'construction']
    for sector in sectors:
        df[f'{sector}_emp'] = df['Qtd Vínculos Ativos'] * df[sector]
        df[f'{sector}_indiv'] = df['Qtd Vínculos Ativos'] * df[sector] * df['individual']
        df[f'{sector}_s'] = df['Qtd Vínculos Ativos'] * df[sector] * df['small']
        df[f'{sector}_m'] = df['Qtd Vínculos Ativos'] * df[sector] * df['medium']
        df[f'{sector}_l'] = df['Qtd Vínculos Ativos'] * df[sector] * df['large']


    df = df.rename(columns={'Município':'ibge_code'})
    df['ibge_code'] = df['ibge_code'].astype(int) 

    vars = ['Ind Atividade Ano', 'small', 'medium', 'large', 'private', 'individual', 'transportation',
       'accommodation', 'retail', 'construction', 'Qtd Vínculos Ativos','transportation_emp',
       'transportation_indiv', 'transportation_s', 'transportation_m',
       'transportation_l', 'accommodation_emp', 'accommodation_indiv',
       'accommodation_s', 'accommodation_m', 'accommodation_l', 'retail_emp',
       'retail_indiv', 'retail_s', 'retail_m', 'retail_l', 'construction_emp',
       'construction_indiv', 'construction_s', 'construction_m',
       'construction_l']

    df = df.groupby(['ibge_code','year'])[vars].sum()
    df.rename(columns={'Ind Atividade Ano':'total','Qtd Vínculos Ativos':'total_emp'}, inplace=True)
     
    globals()[name] = df
    names_list.append(name)
    print('Finished year ' + str(year))

# Use the list of names in _list to call all dfs and concatenate them
rais = pd.concat((globals()[name] for name in names_list), axis=0)
rais.sort_index(inplace=True)

Building year 2006
Finished year 2006
Building year 2007
Finished year 2007
Building year 2008
Finished year 2008
Building year 2009
Finished year 2009
Building year 2010
Finished year 2010
Building year 2011
Finished year 2011
Building year 2012
Finished year 2012
Building year 2013
Finished year 2013
Building year 2014
Finished year 2014
Building year 2015
Finished year 2015
Building year 2016
Finished year 2016
Building year 2017
Finished year 2017


In [6]:
rais

total   small  medium  large  private  individual  \
ibge_code year                                                       
110001    2006     602     729       7      1      490         110   
          2007     498     734       8      1      516         113   
          2008     613     770       4      1      546         113   
          2009     592     749       4      1      554         119   
          2010     630     879       7      1      629         129   
...                ...     ...     ...    ...      ...         ...   
530010    2013  112816  118793    4821   1172   109879        2530   
          2014  108378  119604    4827   1160   109291        2463   
          2015  103829  120268    4650   1114   106008        2379   
          2016  103123  121523    4429   1023   103649        2279   
          2017  103294  122347    4367   1039   101156        2175   

                transportation  accommodation  retail  construction  ...  \
ibge_code year                                                       ...   
110001    2006              17            114     245            11  ...   
          2007              18            113     253            12  ...   
          2008              18            185     263            20  ...   
          2009              22             79     262            22  ...   
          2010              28            131     301            32  ...   
...                        ...            ...     ...           ...  ...   
530010    2013            4345          16849   34425          9378  ...   
          2014            4493          17188   33728          9229  ...   
          2015            4535          17308   33108          8629  ...   
          2016            4537          17417   32430          8174  ...   
          2017            4554          17896   31651          7592  ...   

                retail_emp  retail_indiv  retail_s  retail_m  retail_l  \
ibge_code year                                                           
110001    2006         459             0       411        48         0   
          2007         435             0       415        20         0   
          2008         495             0       495         0         0   
          2009         535             0       514        21         0   
          2010         573             0       548        25         0   
...                    ...           ...       ...       ...       ...   
530010    2013      131919            64     72900     37877     21142   
          2014      131391            69     72582     37253     21556   
          2015      129015            72     72140     36346     20529   
          2016      120967            61     70317     32802     17848   
          2017      118791            58     69045     32807     16939   

                construction_emp  construction_indiv  construction_s  \
ibge_code year                                                         
110001    2006                 0                   0               0   
          2007                17                   8              17   
          2008                26                   4              26   
          2009                47                   0              47   
          2010                44                   1              44   
...                          ...                 ...             ...   
530010    2013             79840                 342           17423   
          2014             69197                 284           16321   
          2015             53427                 277           14169   
          2016             45972                 282           11472   
          2017             44704                 253           10102   

                construction_m  construction_l  
ibge_code year                                  
110001    2006               0               0  
          2007               0               0  
          2008       

In [16]:
df = pd.read_excel('/Users/jpmvbastos/Documents/GitHub/AppliedEconometrics/Causal Inference/TermProject/Data/munic_data.xlsx')
df

,Sigla,Codigo,Município,Year,bankbranches,bankdeposits,homiciderate,icms_transfers,pibmunicipal,savings,...,population,AEROPORTO DE DESTINO (NOME),AEROPORTO DE DESTINO (UF),ANO,ncountry_from,nairports_from,npassengers,ibge_code,host,cand
0,GO,5200050,ABADIA DE GOIÁS,2003,NaN,NaN,17.580872,786765.74,37920.338097,NaN,...,5621.0,NaN,NaN,NaN,0,0,0,520005,0,0
1,GO,5200050,ABADIA DE GOIÁS,2004,NaN,NaN,16.963528,934511.02,36710.710950,NaN,...,6054.0,NaN,NaN,NaN,0,0,0,520005,0,0
2,GO,5200050,ABADIA DE GOIÁS,2005,NaN,NaN,47.664442,928955.40,40500.359822,NaN,...,6294.0,NaN,NaN,NaN,0,0,0,520005,0,0
3,GO,5200050,ABADIA DE GOIÁS,2006,NaN,NaN,15.883100,815089.56,41983.969137,NaN,...,6531.0,NaN,NaN,NaN,0,0,0,520005,0,0
4,GO,5200050,ABADIA DE GOIÁS,2008,NaN,NaN,64.703979,1238956.84,47333.365220,NaN,...,6182.0,NaN,NaN,NaN,0,0,0,520005,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78339,SP,3533809,ÓLEO,2014,NaN,NaN,NaN,2859740.22,56968.400922,NaN,...,2652.0,NaN,NaN,NaN,0,0,0,353380,0,0
78340,SP,3533809,ÓLEO,2015,NaN,NaN,NaN,3032534.69,57057.341856,NaN,...,2628.0,NaN,NaN,NaN,0,0,0,353380,0,0
78341,SP,3533809,ÓLEO,2016,NaN,NaN,NaN,3215613.05,57017.611839,NaN,...,2605.0,NaN,NaN,NaN,0,0,0,353380,0,0
78342,SP,3533809,ÓLEO,2017,NaN,NaN,NaN,3498748.00,54150.480409,NaN,...,2583.0,NaN,NaN,NaN,0,0,0,353380,0,0


In [17]:
data = pd.merge(df, rais, left_on=['ibge_code','Year'], right_on=['ibge_code','year'], how='inner')
data

,Sigla,Codigo,Município,Year,bankbranches,bankdeposits,homiciderate,icms_transfers,pibmunicipal,savings,...,retail_emp,retail_indiv,retail_s,retail_m,retail_l,construction_emp,construction_indiv,construction_s,construction_m,construction_l
0,GO,5200050,ABADIA DE GOIÁS,2006,NaN,NaN,15.883100,815089.56,41983.969137,NaN,...,60,0,60,0,0,12,0,12,0,0
1,GO,5200050,ABADIA DE GOIÁS,2008,NaN,NaN,64.703979,1238956.84,47333.365220,NaN,...,84,0,84,0,0,48,0,48,0,0
2,GO,5200050,ABADIA DE GOIÁS,2009,NaN,NaN,14.562400,1209024.73,47180.369055,NaN,...,89,0,89,0,0,21,0,21,0,0
3,GO,5200050,ABADIA DE GOIÁS,2011,NaN,NaN,27.677830,1784234.53,61714.552997,NaN,...,103,0,76,27,0,60,2,60,0,0
4,GO,5200050,ABADIA DE GOIÁS,2012,NaN,NaN,13.517167,2040858.14,125600.497570,NaN,...,294,0,102,56,136,210,7,90,120,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55668,SP,3533809,ÓLEO,2013,1.0,96.880595,NaN,2810075.00,48136.626094,400.911722,...,13,0,13,0,0,5,0,5,0,0
55669,SP,3533809,ÓLEO,2014,NaN,NaN,NaN,2859740.22,56968.400922,NaN,...,17,0,17,0,0,14,0,14,0,0
55670,SP,3533809,ÓLEO,2015,NaN,NaN,NaN,3032534.69,57057.341856,NaN,...,13,0,13,0,0,40,0,1,39,0
55671,SP,3533809,ÓLEO,2016,NaN,NaN,NaN,3215613.05,57017.611839,NaN,...,16,0,16,0,0,0,0,0,0,0


In [18]:
data.to_csv('/Users/jpmvbastos/Documents/GitHub/AppliedEconometrics/Causal Inference/TermProject/Data/MainData.csv', sep=';' , index=False)

In [9]:
caged = pd.read_csv('/Users/jpmvbastos/Library/CloudStorage/OneDrive-TexasTechUniversity/Fall 2023/Causal Inference/Term Paper/Data/CAGED/CAGEDEST_012013.txt', encoding='latin1', delimiter=';', low_memory=False)

In [11]:
len(caged)

3559644